# Loading EM Images into Tensors

In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
import PIL
import imageio
import libtiff

In [2]:
torch.cuda.set_device(0)

In [14]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
np.set_printoptions(linewidth=110)

## Set path for data

In [3]:
# Modify accordingly
path = Path('/data/alaa/Dropbox (BPHO)/BPHO Staff/USF/EM/')

dataset_name = 'real-world_SEM'

lr_name = f'training/trainsets/crappified/'
lr_path = path/f'{lr_name}'
lr_files = list(lr_path.glob('*.tif'))

hr_name = f'training/trainsets/hr/'
hr_path = path/f'{hr_name}'
hr_files = list(hr_path.glob('*.tif'))

print('Processing '+str(len(lr_files))+' files...')

Processing 67157 files...


# Load Images

In [4]:
def load_img(img_fn):
    """Loads input image into matrix using filename"""
    img = libtiff.TiffFile(img_fn)
    img_mat = img.get_tiff_array()[0].astype(np.float32)[np.newaxis, :]
    return img_mat

In [5]:
# sort filenames so that they are aligned for visualization
lr_files = sorted(lr_files)
hr_files = sorted(hr_files)

In [6]:
lr_files[0], hr_files[1]

(PosixPath('/data/alaa/Dropbox (BPHO)/BPHO Staff/USF/EM/training/trainsets/crappified/EM_train_lr_00002.tif'),
 PosixPath('/data/alaa/Dropbox (BPHO)/BPHO Staff/USF/EM/training/trainsets/hr/EM_train_hr_00002.tif'))

In [7]:
# Set size for visualizations
fig_size = plt.rcParams["figure.figsize"]  # Get current size
print("Current size:", fig_size)
 
# Set figure width to 12 and height to 9
fig_size[0] = 15
fig_size[1] = 12
plt.rcParams["figure.figsize"] = fig_size

Current size: [6.0, 4.0]


# On the Use of SVD as a Potential Crappification Method

In [8]:
# This method computes the right singular vectors (v) and singular values 6t7of the input matrix A
def compute_right_singular_vals(A):
    ATA = A.T @ A
    eigvals, eigvecs = np.linalg.eigh(ATA)
    V = np.flip(eigvecs, 1)
    S = np.sqrt(np.flip(eigvals))
    return S, V

In [9]:
# This method computes the left singular vectors (v) of the input matrix A
def compute_left_singular_vals(A, S, V, m, n):
    U = np.zeros((m,m))
    num_iters = min(m, n)
    for j in range(num_iters):
        U[:, j] = A @ V[:, j] / S[j]
    return U

In [10]:
# this method computes a reconstructe image using the best rank k approximation 
def reconstruct_img(U, S, V, k):
    img = 0
    for j in range(k):
        img += S[j] * np.outer(U[:, j], V[:, j])
    return img

In [11]:
# this method computes the singular value decomposition of an input matrix
def compute_svd(img, k=20):
    # convert image array to float
    A = np.squeeze(img.astype(float))
    # get image dimensions
    m, n = A.shape
    # compute right singular vectors and the singular values
    S, V = compute_right_singular_vals(A)
    # compute left singular vectors
    U = compute_left_singular_vals(A, S, V, m, n)
    S = np.nan_to_num(S)
    # reconstruct input using rank k best approximation
    compressed_img = reconstruct_img(U, S, V, k)
    return compressed_img

In [12]:
def visualize_svd(lr_fn, hr_fn, lr_k=50, hr_k=25):
    lr_im = load_img(lr_fn) # load image to tensor
    hr_im = load_img(hr_fn)
    f, axarr = plt.subplots(2,2) # create visualizations
    axarr[0][0].imshow(np.squeeze(hr_im), cmap=plt.cm.gray) # visualize image tensor
    axarr[0][0].set_title("HR")
    axarr[0][1].imshow(compute_svd(hr_im, k=hr_k), cmap=plt.cm.gray) # visualize image tensor
    axarr[0][1].set_title("HR SVD")
    axarr[1][0].imshow(np.squeeze(lr_im), cmap=plt.cm.gray) # visualize original image file
    axarr[1][0].set_title("LR")
    axarr[1][1].imshow(compute_svd(lr_im, k=lr_k), cmap=plt.cm.gray) # visualize original image file
    axarr[1][1].set_title("LR SVD")
    plt.show() # show visualization

In [15]:
@interact
def show_sample(sample=10):
    return visualize_svd(lr_files[sample], hr_files[sample], lr_k=50, hr_k=25)

interactive(children=(IntSlider(value=10, description='sample', max=30, min=-10), Output()), _dom_classes=('wi…

# Here we Perform Image Crappification

In [ ]:
class Crap:
    """This is Crap: the embodiment of the crappification process. 
        Parameters should consist of crapifying methods 
        e.g. SVD, gaussian blur, Neural-Network-based noise predictions."""
    
    def __init__(self, crappifier, config={'default': int}):
        
        # your crappify method of choice
        self.crapify = crappifier
        # contains hyperparameters e.g. number of singular values
        self.config = config
        
    def fit(self, X):
    "Crappify your images"
        return self.crapify(X, settings=self.config)
    
    def __del__(self):
    "Destroy process and reinitialize"
        raise NotImplemented
        
    def SVD(self):
    "Apply Singular Value Decomposition to dataset"
    
        #TODO: Linear Algebra hw_final <-- but pick random singular values!
        raise NotImplemented
    
    def gaus_blur(self):
    "Apply Gaussian Blur to dataset"
    
    #TODO: Annette's notebook <-- but make it more random!
    raise NotImplemented
    
    def s_p(self):
    "Apply salt & pepper noise to dataset"
    
    #TODO: It's already randomized; that's good!
    raise NotImplemented
    
    def model(self):
    "Apply predictions from another neural network"
    
    #This one might be too difficult to do, but would be interesting to see
    raise NotImplemented
    
    def visualize(self):
    "Show visual inspection of LR-HR image pairs"
    
    # This has been implemented in another notebook
    # TODO: Refactor visualization notebook 
    raise NotImplemented